In [ ]:
#!wget https://git.ai.ssau.ru/tk/big_data/raw/branch/master/data/posts_sample.xml

In [19]:
#!wget https://git.ai.ssau.ru/tk/big_data/raw/branch/master/data/programming-languages.csv

In [20]:
#!pip install pyspark

In [21]:
import pyspark
import os
from datetime import datetime
from pyspark.sql.functions import col
from pyspark.sql import SparkSession

In [22]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.13.0 pyspark-shell'
SparkSes = SparkSession.builder.appName("l2").master("local[*]").getOrCreate()

In [23]:
progLang = SparkSes.read.csv("programming-languages.csv")
progLang_list = [str(x[0]) for x in progLang.collect()]

In [24]:
posts = SparkSes.read.format("xml").options(rowTag="row").load('posts_sample.xml')

In [25]:
def languageDetection(x):
  tag = None
  for language in progLang_list:
    if "<" + language.lower() + ">" in x._Tags.lower():
      tag = language
      break
  if tag is None:
    return None
  return (x._Id, tag)


def checkDate(x, year):
  CreationDate = x._CreationDate
  return CreationDate >= datetime(year=year, month=1, day=1) and\
         CreationDate <= datetime(year=year, month=12, day=31)

In [26]:
res = {}

for year in range(2010, 2020):
  res[year] = posts.rdd\
      .filter(lambda x: x._Tags is not None and checkDate(x, year))\
      .map(languageDetection).filter(lambda x: x is not None)\
      .keyBy(lambda x: x[1])\
      .aggregateByKey(0, lambda x, y: x + 1, lambda x1, x2: x1 + x2,)\
      .sortBy(lambda x: x[1], ascending=False).toDF()

  res[year] = res[year].select(col("_1").alias("Язык программирования"),
                               col("_2").alias(f"Количество упоминаний в {year}")).limit(10)
  res[year].show()

+---------------------+----------------------------+
|Язык программирования|Количество упоминаний в 2010|
+---------------------+----------------------------+
|                 Java|                          52|
|           JavaScript|                          44|
|                  PHP|                          42|
|               Python|                          25|
|          Objective-C|                          22|
|                    C|                          20|
|                 Ruby|                          11|
|               Delphi|                           7|
|                    R|                           3|
|                 Bash|                           3|
+---------------------+----------------------------+

+---------------------+----------------------------+
|Язык программирования|Количество упоминаний в 2011|
+---------------------+----------------------------+
|                  PHP|                          97|
|                 Java|                      

# Новый раздел

In [ ]:
for year in res.keys():
    res[year].write.format("parquet").save(f"{year}")